# Practice Session 04: Basket analysis

Author: <font color="blue">Miguel Rando Ramirez</font>

E-mail: <font color="blue">miguel_rando01@estudiant.upf.edu</font>

Date: <font color="blue">15/10/2024</font>

In [97]:
import numpy as np  
import matplotlib.pyplot as plt  
import pandas as pd  
import csv
import gzip

from apyori import apriori

# 1. Playing with apyori

In [98]:
# LEAVE AS-IS

def print_apyori_output (association_results, info=False, info_key=False):
    for relation_record in association_results:
        itemset = list(relation_record.items)
        
        # Consider only itemsets of two elements
        if len(itemset) > 1: 
        
            print("Rules involving itemset %s" % itemset)
            support = relation_record.support

            for rules in relation_record.ordered_statistics:
                antecedent = list(rules.items_base)
                consequent = list(rules.items_add)
                
                if info_key:
                    antecedent = [info.loc[x][info_key] for x in antecedent]
                    consequent = [info.loc[x][info_key] for x in consequent]
                
                confidence = rules.confidence
                lift = rules.lift

                print("%s => %s (support=%.4f, confidence=%.2f, lift=%.2f)" %
                      (antecedent, consequent, support, confidence, lift))
            print()

<font size="+1" color="red">Replace this cell with your own example of transactions (at least 20 transactions). Execute the apriori algorithm, in which you should obtain at least <strong>two</strong> rules of the form ['A', 'B'] => ['C'], i.e., at least two rules having a 2-itemset in the antecedent and a 1-itemset in the consequent. Modify the transactions until you obtain such rules.</font>



In [99]:
transactions = [
    ['Milk', 'Bread', 'Butter'],
    ['Beer', 'Olives', 'Chips'],
    ['Milk', 'Olives', 'Beer', 'Bread'],
    ['Bread', 'Butter', 'Beer'],
    ['Milk', 'Olives', 'Butter'],
    ['Milk', 'Bread'],
    ['Olives', 'Chips', 'Soda'],
    ['Milk', 'Bread', 'Butter', 'Soda'],
    ['Bread', 'Butter', 'Chips'],
    ['Milk', 'Bread', 'Chips', 'Butter'],
    ['Olives', 'Milk', 'Bread'],
    ['Olives', 'Beer'],
    ['Milk', 'Chips'],
    ['Bread', 'Beer', 'Chips'],
    ['Butter', 'Chips'],
    ['Milk', 'Bread', 'Butter', 'Beer'],
    ['Olives', 'Soda'],
    ['Olives', 'Bread', 'Butter'],
    ['Milk', 'Beer', 'Butter'],
    ['Chips', 'Soda', 'Bread'],
    ['Milk', 'Bread', 'Beer']  
]


results = list(apriori(transactions, min_support=0.1, min_confidence=0.5, min_lift=1.0, min_length=2))
print_apyori_output(results)



Rules involving itemset ['Beer', 'Bread']
['Beer'] => ['Bread'] (support=0.2381, confidence=0.62, lift=1.01)

Rules involving itemset ['Butter', 'Bread']
['Bread'] => ['Butter'] (support=0.3333, confidence=0.54, lift=1.13)
['Butter'] => ['Bread'] (support=0.3333, confidence=0.70, lift=1.13)

Rules involving itemset ['Bread', 'Milk']
['Bread'] => ['Milk'] (support=0.3810, confidence=0.62, lift=1.17)
['Milk'] => ['Bread'] (support=0.3810, confidence=0.73, lift=1.17)

Rules involving itemset ['Butter', 'Milk']
['Butter'] => ['Milk'] (support=0.2857, confidence=0.60, lift=1.15)
['Milk'] => ['Butter'] (support=0.2857, confidence=0.55, lift=1.15)

Rules involving itemset ['Beer', 'Bread', 'Milk']
['Beer', 'Bread'] => ['Milk'] (support=0.1429, confidence=0.60, lift=1.15)
['Beer', 'Milk'] => ['Bread'] (support=0.1429, confidence=0.75, lift=1.21)

Rules involving itemset ['Butter', 'Bread', 'Milk']
['Butter', 'Bread'] => ['Milk'] (support=0.1905, confidence=0.57, lift=1.09)
['Bread', 'Milk'] =>

<font size="+1" color="red">Replace this cell with a markdown cell containing (1) a printout of the rules you have obtained, and (2) for each of those rules, indicate clearly how the support, confidence, and lift is calculated. Do not merely repeat the formula: indicate how each number is computed based on the transactions you provided, as if you were trying to verify that the numbers are correct.</font>

I dont know why it appear some rules with only one rule in the antecedent if I have set the min_length of the antecedent to be 2. Any way we have this rules:


Rules involving itemset ['Beer', 'Bread', 'Milk'] 

['Beer', 'Bread'] => ['Milk'] (support=0.1429, confidence=0.60, lift=1.15) 

['Beer', 'Milk'] => ['Bread'] (support=0.1429, confidence=0.75, lift=1.21)


Rules involving itemset ['Butter', 'Bread', 'Milk'] 

['Butter', 'Bread'] => ['Milk'] (support=0.1905, confidence=0.57, lift=1.09) 

['Bread', 'Milk'] => ['Butter'] (support=0.1905, confidence=0.50, lift=1.05) 

['Butter', 'Milk'] => ['Bread'] (support=0.1905, confidence=0.67, lift=1.08)

Im only going to explain these two rules:

['Beer', 'Bread'] => ['Milk']

Support:
There are 3 transactions that have those 3 items. Then 3/21 = 0.1429 (21 is the total num of transactions)
Confidence:
There are 5 rules that contain the antecedent (beer, bread). So we divide the transaction where the 3 appear between the number of rules where the antecedent appear. Then 3/5 = 0.6 
Lift: 
Here we calculate the support of milk. 8/21 = 0.38
Then the lift is the confidence between the support of milk. 0.6/0.38 = 1.15


['Butter', 'Milk'] => ['Bread'] 

Support:
There are 4 transactions that have those 3 items. Then 4/21 = 0.19 
Confidence:
There are 6 rules that contain the antecedent (butter, milk). So we divide the transactions where the 3 appear between the number of rules where the antecedent appear. Then 4/6 = 0.666 
Lift: 
Here we calculate the support of Bread. 13/21 = 0.62
Then the lift is the confidence between the support of bread. 0.67/0.62 = 1.08

# 2. Load and prepare the shopping baskets

In [100]:
# LEAVE AS-IS

# File names
INPUT_PRODUCTS = "instacart-products.csv"
INPUT_TRANSACTIONS = "instacart-transactions.csv.gz"

# Read into a dataframe
products = pd.read_csv(INPUT_PRODUCTS, delimiter=",")

# Set product_id as index, and drop column aisle_id
products = products.set_index('product_id').drop(columns=['aisle_id'])

products.head(100)

,product_name,department_id
product_id,,
1,Chocolate Sandwich Cookies,19
2,All-Seasons Salt,13
3,Robust Golden Unsweetened Oolong Tea,7
4,Smart Ones Classic Favorites Mini Rigatoni Wit...,1
5,Green Chile Anytime Sauce,13
...,...,...
96,Sprinklez Confetti Fun Organic Toppings,13
97,Organic Chamomile Lemon Tea,7
98,2% Yellow American Cheese,16


## 2.1. Select by department

In [101]:
# LEAVE AS-IS

DEPT_BAKERY = 3
DEPT_VEGGIES = 4
DEPT_ALCOHOL = 5
DEPT_WORLD = 6
DEPT_DRINKS = 7
DEPT_PETS = 8
DEPT_PHARMACY = 11
DEPT_CLEANING = 17
DEPT_BABIES = 18

<font size="+1" color="red">Replace this cell with your code for *select_from_departments*.</font>

In [102]:
def select_from_departments(products, product_ids, department_ids):
    selected_product_ids = []
    
    # Iterate through the list of product IDs
    for product_id in product_ids:
        if products.loc[product_id]['department_id'] in department_ids:
            selected_product_ids.append(product_id)
    
    return selected_product_ids

<font size="+1" color="red">Replace this cell with code to test your function with three different test cases. Each test case is a list of items and a list of 1, 2, or 3 departments.</font>

In [103]:
# Test 1: 
test_1_product_ids = [1, 2, 3, 4, 5, 6]  
test_1_department_ids = [DEPT_BAKERY]  # Only Bakery department
result_1 = select_from_departments(products, test_1_product_ids, test_1_department_ids)
print("Test 1 Products:", result_1)

# Test 2: This is the one provided before to check that the result is okay
test_2_product_ids = [22, 26, 45, 54, 57, 71, 111, 112]
test_2_department_ids = [DEPT_BAKERY, DEPT_CLEANING]  
result_2 = select_from_departments(products, test_2_product_ids, test_2_department_ids)
print("Test 2 Products:", result_2)

# Test 3: 
test_3_product_ids = [13, 14, 15, 16, 17, 18]  
test_3_department_ids = [DEPT_ALCOHOL, DEPT_CLEANING, DEPT_PETS]  # Alcohol, Cleaning, and Pets departments
result_3 = select_from_departments(products, test_3_product_ids, test_3_department_ids)
print("Test 3 Products:", result_3)

Test 1 Products: []
Test 2 Products: [57, 71, 111]
Test 3 Products: [14]


## 2.2. Read and filter transactions

<font size="+1" color="red">Replace this cell with your code to read transactions, keeping only items in DEPT_PHARMACY. Remember to stop after storing 5000 of the transactions read.</font>

In [104]:
transactions = []

department_ids = [DEPT_PHARMACY]  

# Open a compressed file
with gzip.open(INPUT_TRANSACTIONS, "rt") as inputfile:
    
    # Create a CSV reader
    reader = csv.reader(inputfile, delimiter=",")
    
    # Initialize a counter for transactions read
    transactions_read = 0

    # Iterate through the CSV file
    for row in reader:
        
        # Convert to integers
        items = [int(x) for x in row]
        
        # Filter the items based on department
        filtered_items = select_from_departments(products, items, department_ids)
        
        # If we have filtered items, add to transactions
        if filtered_items:
            transactions.append(filtered_items)
        
        # Increment the transaction read counter
        transactions_read += 1
        
        # Every 1000 transactions read, print the count
        if transactions_read % 1000 == 0:
            print(f"Transactions read: {transactions_read}, Transactions stored: {len(transactions)}")
        
        # Stop if we have stored 5000 transactions
        if len(transactions) >= 5000:
            break

print(f"Total transactions read: {transactions_read}, Total transactions stored: {len(transactions)}")

Transactions read: 1000, Transactions stored: 102
Transactions read: 2000, Transactions stored: 219
Transactions read: 3000, Transactions stored: 313
Transactions read: 4000, Transactions stored: 422
Transactions read: 5000, Transactions stored: 526
Transactions read: 6000, Transactions stored: 619
Transactions read: 7000, Transactions stored: 722
Transactions read: 8000, Transactions stored: 810
Transactions read: 9000, Transactions stored: 920
Transactions read: 10000, Transactions stored: 1046
Transactions read: 11000, Transactions stored: 1152
Transactions read: 12000, Transactions stored: 1245
Transactions read: 13000, Transactions stored: 1352
Transactions read: 14000, Transactions stored: 1459
Transactions read: 15000, Transactions stored: 1559
Transactions read: 16000, Transactions stored: 1663
Transactions read: 17000, Transactions stored: 1753
Transactions read: 18000, Transactions stored: 1854
Transactions read: 19000, Transactions stored: 1943
Transactions read: 20000, Tran

## 2.3. Extract association rules and comment on them

<font size="+1" color="red">Replace this cell with your code to extract association rules from the read transactions.</font>

In [105]:
results = list(apriori(transactions, min_support=0.001, min_confidence=0.1, min_lift=0.6))
print_apyori_output(results, products, 'product_name')

Rules involving itemset [16825, 1406]
['Vegan Nutritional Shake Sweet Vanilla Bean'] => ['Vegan Smooth Chocolate Nutritional Shake'] (support=0.0012, confidence=0.40, lift=80.00)
['Vegan Smooth Chocolate Nutritional Shake'] => ['Vegan Nutritional Shake Sweet Vanilla Bean'] (support=0.0012, confidence=0.24, lift=80.00)

Rules involving itemset [7976, 16019]
['One Plant-Based Chocolate Flavor Nutritional Shake Drink Mix'] => ['All-In-One French Vanilla Nutritional Shake Sachet'] (support=0.0010, confidence=0.31, lift=97.66)
['All-In-One French Vanilla Nutritional Shake Sachet'] => ['One Plant-Based Chocolate Flavor Nutritional Shake Drink Mix'] (support=0.0010, confidence=0.31, lift=97.66)

Rules involving itemset [27544, 39162]
['Lemon Verbena Hand Soap'] => ['Clean Day Basil Hand Soap'] (support=0.0018, confidence=0.12, lift=11.21)
['Clean Day Basil Hand Soap'] => ['Lemon Verbena Hand Soap'] (support=0.0018, confidence=0.16, lift=11.21)



<font size="+1" color="red">Replace this cell with a brief commentary on what you would recommend to the shopping app considering the extracted association rules.</font>

In the first rule we can see that the 40% of the people who buy the chocolate shake, also buy the vanilla shake. So they can offer these shakes as a combo pack or provide discounts on the second product to increase sales.
The second rule is a similar case.
At the last rule we can see that the 16% of the people who buy the Basil Soap also buy the lemon one. And I would do the same.

Also, of course, I would put this items close between them in the store.


## 2.4. Extract association rules and comment on them (other departments)

In [106]:
transactions = []

department_ids = [DEPT_BAKERY, DEPT_ALCOHOL]  

# Open a compressed file
with gzip.open(INPUT_TRANSACTIONS, "rt") as inputfile:
    
    # Create a CSV reader
    reader = csv.reader(inputfile, delimiter=",")
    
    # Initialize a counter for transactions read
    transactions_read = 0

    # Iterate through the CSV file
    for row in reader:
        
        # Convert to integers
        items = [int(x) for x in row]
        
        # Filter the items based on department
        filtered_items = select_from_departments(products, items, department_ids)
        
        # If we have filtered items, add to transactions
        if filtered_items:
            transactions.append(filtered_items)
        
        # Increment the transaction read counter
        transactions_read += 1
        
        # Every 1000 transactions read, print the count
        if transactions_read % 1000 == 0:
            print(f"Transactions read: {transactions_read}, Transactions stored: {len(transactions)}")
        
        # Stop if we have stored 5000 transactions
        if len(transactions) >= 5000:
            break

print(f"Total transactions read: {transactions_read}, Total transactions stored: {len(transactions)}")

Transactions read: 1000, Transactions stored: 289
Transactions read: 2000, Transactions stored: 574
Transactions read: 3000, Transactions stored: 906
Transactions read: 4000, Transactions stored: 1202
Transactions read: 5000, Transactions stored: 1502
Transactions read: 6000, Transactions stored: 1799
Transactions read: 7000, Transactions stored: 2091
Transactions read: 8000, Transactions stored: 2392
Transactions read: 9000, Transactions stored: 2692
Transactions read: 10000, Transactions stored: 3002
Transactions read: 11000, Transactions stored: 3252
Transactions read: 12000, Transactions stored: 3545
Transactions read: 13000, Transactions stored: 3839
Transactions read: 14000, Transactions stored: 4143
Transactions read: 15000, Transactions stored: 4413
Transactions read: 16000, Transactions stored: 4718
Transactions read: 17000, Transactions stored: 4997
Total transactions read: 17006, Total transactions stored: 5000


In [107]:
results = list(apriori(transactions, min_support=0.001, min_confidence=0.1, min_lift=0.6))
print_apyori_output(results, products, 'product_name')

Rules involving itemset [2120, 41131]
['Sauvignon Blanc'] => ['India Pale Ale'] (support=0.0012, confidence=0.13, lift=20.20)
['India Pale Ale'] => ['Sauvignon Blanc'] (support=0.0012, confidence=0.18, lift=20.20)

Rules involving itemset [17706, 5077]
['Organic Whole Grain Wheat English Muffins'] => ['100% Whole Wheat Bread'] (support=0.0018, confidence=0.11, lift=1.68)

Rules involving itemset [42460, 5077]
['Plain Mini Bagels'] => ['100% Whole Wheat Bread'] (support=0.0010, confidence=0.14, lift=2.07)

Rules involving itemset [5077, 48183]
['Flour Tortillas'] => ['100% Whole Wheat Bread'] (support=0.0010, confidence=0.19, lift=2.87)

Rules involving itemset [10673, 17341]
['Presliced Everything Bagels'] => ['Original Nooks & Crannies English Muffins'] (support=0.0014, confidence=0.16, lift=9.36)

Rules involving itemset [20682, 17341]
['Presliced Everything Bagels'] => ['Cinnamon Raisin Swirl Pre-Sliced Bagels'] (support=0.0012, confidence=0.14, lift=21.31)
['Cinnamon Raisin Swirl P

We can see that between these two departaments, there are no relation between the porducts.
The rules obtained are between products of the same departament. Then, we see that bakery most of the rules are bakery products. That is normal as it is easier to buy different pieces of bakery, while wen people is going to buy alcohol, usually only buy one item. Any way the Pale Ale is related with the Sauvignon blanc and with the beer.

<font size="+2" color="#003300">I hereby declare that, except for the code provided by the course instructors, all of my code, report, and figures were produced by myself.</font>